<a href="https://colab.research.google.com/github/SJinLee/cryptology/blob/main/05.AES_GF256.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.cs.uaf.edu/2015/spring/cs463/lecture/03_23_AES.html

In [4]:
def to_binary_list(m,ndigits=8):
    b = []
    for i in range(ndigits):
        b = [m%2] + b
        m = m//2
    return b
def to_num(b):
    m = 0
    for n in b:
        m = m*2+n
    return m

In [5]:
def add_gf256(a,b):
    return a^b
def mult_gf256(a,b):
    irr = 0x11b
    high = 0x100
    c = 0
    for bit in range(7,-1,-1):
        if b&(1<<bit):
            c=c^(a<<bit)

    for bit in range(7,-1,-1):
        if c & (high<<bit):
            c=c^(irr<<bit)
    return c

In [6]:
a = 0x57
b = 0x83
c = add_gf256(a,b)
hex(c)

'0xd4'

In [7]:
a = 0x57
b = 0x83
c = mult_gf256(a,b)
hex(c)

'0xc1'

In [8]:
to_binary_list(c)

[1, 1, 0, 0, 0, 0, 0, 1]

In [9]:
def gmix_column(r):
    a = [0]*4;
    b = [0]*4;
    # The array 'a' is simply a copy of the input array 'r'
    # The array 'b' is each element of the array 'a' multiplied by 2
    # in Rijndael's Galois field
    # a[n] ^ b[n] is element n multiplied by 3 in Rijndael's Galois field
    for c in range(4):
        a[c] = r[c]
        h = r[c] >> 7
        b[c] = r[c] << 1
        b[c] ^= h * 0x1B # Rijndael's Galois field
    r[0] = b[0] ^ a[3] ^ a[2] ^ b[1] ^ a[1] # 2 * a0 + a3 + a2 + 3 * a1
    r[1] = b[1] ^ a[0] ^ a[3] ^ b[2] ^ a[2] # 2 * a1 + a0 + a3 + 3 * a2
    r[2] = b[2] ^ a[1] ^ a[0] ^ b[3] ^ a[3] # 2 * a2 + a1 + a0 + 3 * a3
    r[3] = b[3] ^ a[2] ^ a[1] ^ b[0] ^ a[0] # 2 * a3 + a2 + a1 + 3 * a0
    return r

In [10]:
c = [0x01,0x02,0x03,0x04]
gmix_column(c)

[3, 4, 9, 10]

In [37]:
def calc_all_inv():
    inv = []
    for a in range(256):
        for b in range(256):
            c = mult_gf256(a,b)
            if c==1:
                break
        if c==1:
            inv.append(b)
        else:
            inv.append(0)
    return inv

In [38]:
inv = calc_all_inv()
[' '.join([x.to_bytes(1,'big').hex() for x in inv[i:i+16]]) for i in range(0,256,16)]

['00 01 8d f6 cb 52 7b d1 e8 4f 29 c0 b0 e1 e5 c7',
 '74 b4 aa 4b 99 2b 60 5f 58 3f fd cc ff 40 ee b2',
 '3a 6e 5a f1 55 4d a8 c9 c1 0a 98 15 30 44 a2 c2',
 '2c 45 92 6c f3 39 66 42 f2 35 20 6f 77 bb 59 19',
 '1d fe 37 67 2d 31 f5 69 a7 64 ab 13 54 25 e9 09',
 'ed 5c 05 ca 4c 24 87 bf 18 3e 22 f0 51 ec 61 17',
 '16 5e af d3 49 a6 36 43 f4 47 91 df 33 93 21 3b',
 '79 b7 97 85 10 b5 ba 3c b6 70 d0 06 a1 fa 81 82',
 '83 7e 7f 80 96 73 be 56 9b 9e 95 d9 f7 02 b9 a4',
 'de 6a 32 6d d8 8a 84 72 2a 14 9f 88 f9 dc 89 9a',
 'fb 7c 2e c3 8f b8 65 48 26 c8 12 4a ce e7 d2 62',
 '0c e0 1f ef 11 75 78 71 a5 8e 76 3d bd bc 86 57',
 '0b 28 2f a3 da d4 e4 0f a9 27 53 04 1b fc ac e6',
 '7a 07 ae 63 c5 db e2 ea 94 8b c4 d5 9d f8 90 6b',
 'b1 0d d6 eb c6 0e cf ad 08 4e d7 e3 5d 50 1e b3',
 '5b 23 38 34 68 46 03 8c dd 9c 7d a0 cd 1a 41 1c']

In [39]:
def affine(b):
    mat = [[1,0,0,0,1,1,1,1],
           [1,1,0,0,0,1,1,1],
           [1,1,1,0,0,0,1,1],
           [1,1,1,1,0,0,0,1],
           [1,1,1,1,1,0,0,0],
           [0,1,1,1,1,1,0,0],
           [0,0,1,1,1,1,1,0],
           [0,0,0,1,1,1,1,1]]
    const = [1,1,0,0,0,1,1,0]
    bb = to_binary_list(b)[::-1]
    cc = []
    for i in range(8):
        c = sum([x*y for x,y in zip(mat[i],bb)]) + const[i]
        cc = cc + [c % 2]
    return to_num(cc[::-1])

In [43]:
sbox = {i:affine(x) for i,x in enumerate(inv)}

In [44]:
[' '.join([sbox[x].to_bytes(1,'big').hex() for x in range(i,i+16)]) for i in range(0,256,16)]

['63 7c 77 7b f2 6b 6f c5 30 01 67 2b fe d7 ab 76',
 'ca 82 c9 7d fa 59 47 f0 ad d4 a2 af 9c a4 72 c0',
 'b7 fd 93 26 36 3f f7 cc 34 a5 e5 f1 71 d8 31 15',
 '04 c7 23 c3 18 96 05 9a 07 12 80 e2 eb 27 b2 75',
 '09 83 2c 1a 1b 6e 5a a0 52 3b d6 b3 29 e3 2f 84',
 '53 d1 00 ed 20 fc b1 5b 6a cb be 39 4a 4c 58 cf',
 'd0 ef aa fb 43 4d 33 85 45 f9 02 7f 50 3c 9f a8',
 '51 a3 40 8f 92 9d 38 f5 bc b6 da 21 10 ff f3 d2',
 'cd 0c 13 ec 5f 97 44 17 c4 a7 7e 3d 64 5d 19 73',
 '60 81 4f dc 22 2a 90 88 46 ee b8 14 de 5e 0b db',
 'e0 32 3a 0a 49 06 24 5c c2 d3 ac 62 91 95 e4 79',
 'e7 c8 37 6d 8d d5 4e a9 6c 56 f4 ea 65 7a ae 08',
 'ba 78 25 2e 1c a6 b4 c6 e8 dd 74 1f 4b bd 8b 8a',
 '70 3e b5 66 48 03 f6 0e 61 35 57 b9 86 c1 1d 9e',
 'e1 f8 98 11 69 d9 8e 94 9b 1e 87 e9 ce 55 28 df',
 '8c a1 89 0d bf e6 42 68 41 99 2d 0f b0 54 bb 16']

In [42]:
def RotWord(w):
    return w[1:] + w[:1]

In [45]:
def SubBytes(b):
    return sbox[b]
def SubWords(w):
    return [SubBytes[x] for x in w]

In [51]:
def get_rcon(Nr):
    rcon = [0,1,2]
    for i in range(3,Nr+1):
        x = mult_gf256(rcon[i-1],2)
        rcon.append(x)
    return rcon

In [56]:
rcon = get_rcon(14)
' '.join([x.to_bytes(1,'big').hex() for x in rcon])

'00 01 02 04 08 10 20 40 80 1b 36 6c d8 ab 4d'